In [1]:
import numpy as np
import os
import pandas as pd
# from sentence_transformers import SentenceTransformer, util
import re
# from sklearn.metrics.pairwise import cosine_similarity

import csv
import pickle
import time


# import tensorflow as tf

In [2]:

def cleaning_data(qc_data):
    app["description"] = app["description"].apply(lambda x : re.sub(r'(<)(.*?)(>)','',x))#去除<.*>标签 
    app["description"] = app["description"].apply(lambda x : re.sub(r'(【)(.*?)(】)','',x))#去除<.*>标签 ，应该还可以加一些括号类！！《
    app["description"] = app["description"].apply(lambda x : re.sub(r'(《)(.*?)(》)','',x))
#     app['description'] = app['description'].str.replace('=-', ' ')
#     app['description'] = app['description'].str.replace('=---', ' ')
#     app['description'] = app['description'].str.replace('=+', ' ')
#     app['description'] = app['description'].str.replace('=--', ' ')
#     app['description'] = app['description'].str.replace('=', ' ')
    app["description"] = app["description"].apply(lambda x : re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))#去除https：wwww.xxxx.com
    app["description"] = app["description"].apply(lambda x : re.sub(r'1\d{10}|(\d{3,4})-(\d{7,8})-\d{3,4}|\d{3,4}-\d{7,8}|\d{9}|\d{8}|\d{10}', '', x))#去除手机号类
    app["description"] = app["description"].apply(lambda x : re.sub(r'\w+([-+.]\w+)*@\w+([-.]\w+)*.\w+([-.]\w+)*.','', x))#去除邮箱
    app["description"] = app["description"].apply(lambda x : re.sub('|'.join(qc_data),'.',x))#去除特殊字符
    return app

def segmenting_sentences(fj_data):
    app1 = app.assign(description=app.description.str.split('['+''.join(fj_data)+']')).explode('description')#切分句子，切分后不会保留切分符号
    app1.reset_index()
    #有一些\n
    return app1

def remove_redundant_lines(dy_data,bh_data):
    # 替换\n换行符及空格，返回非空格的行，即去除空格行：
    # dropna无法使用原因是因为处理后为中文字符串的空格并非nan
    app2 = app1.assign(description=app1.description.str.replace('\n','').replace('  ','').str.strip())#将换行符和空格替换为空
    app2= app2[app2["description"].str.contains('[\d\w]+')] # 匹配任意数字或字符
    app2 = app2[~app2['description'].isin(dy_data)] # 去掉description等于dy_data列表中元素的行
    app2 = app2[~app2["description"].str.contains('|'.join(bh_data))] # 去掉description包含dy_data列表中元素的行
#     app2[app2["description"].map(len)>5]
    app2=app2[app2["description"].str.len()>5] # 长度小于5的不要了(因为有些分完很短感觉没用)
    #    app2 = app["description"].str.apply(lambda x: len(x) )  不知道用lambda x会不会速度快一点？？？？？？
    return app2
def save_files(app2,save_name):
    app2.to_csv(save_name,encoding='utf-8-sig')


In [4]:

app_data = pd.read_csv('app2000.csv' )
app = app_data.iloc[0:1, :]
type(app["description"])


pandas.core.series.Series

In [3]:
app_data = pd.read_csv('app2000.csv' )

# app_data = app_data.loc[5000:7000]

row_num = app_data.shape[0]
save_num = 1000 #没次处理的行数（自定义，觉得一次一万行应该可以），防止死机，这样每次都新存一个csv，最终拼接起来？

for start_index,end_index in zip(range(0,row_num,save_num),range(save_num,row_num+save_num,save_num)):
    app = app_data.iloc[start_index:end_index, :]

    # # 第一步：清洗数据
    qc_data = ['&gt', '邮箱：', '■', '=-','💎','💖','🏆', '👧🏻','💄','🌟','✨','⚽','🔥'
               ,'⭐','__________________________________________','—','——','▲','◆','・','✔','='
               ,'=-','=---','=--','=+','=----']  # 列表1：把想要去除的字符串放到这个列表中去除, (去除字符，not whole sentence, unlike below，比如 “邮箱”这两个字很常见，
    app = cleaning_data(qc_data)

    # # 第二步：分句处理
    fj_data = [';', '!', '?', '●', '•', '！', '!', '*', '✓', '【', '；', '✓', '★','。','.','✔️','=+','=-','=---','➤','®']  # 列表2：（还需补充,用来分割的符号）把想要变成句号的特殊放在这个列表中进行替换，
    app1 = segmenting_sentences(fj_data)

    # # 第三步：去掉多余行
    dy_data = ['视觉革新', '品质升级','·&nbsp','Facebook:','Twitter:','Website:','updates','youtube']  # 列表3：若切分好句子后的description值为列表元素值，则删除该行，（delete whole sentence，必须精确匹配，unlike above）
    bh_data = ['取消订阅：', '客服热线','iPad','神马？','com/','」','iPhone','订阅','穿越火线','Instagram']  # 列表4：若切分好句子后的description值“”包含""列表元素值，则删除该行，（if contain this, delete whole sentence/row）
    app2 = remove_redundant_lines(dy_data,bh_data)

    # # 第四步：保存文件
    save_files(app2,save_name=f'{start_index}'+'-'+f'{end_index-1}.csv')# 文件名为0-10000、10001-20000

C:\Users\Justin\AppData\Local\Temp\ipykernel_2604\2006770839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app["description"] = app["description"].apply(lambda x : re.sub(r'(<)(.*?)(>)','',x))#去除<.*>标签
C:\Users\Justin\AppData\Local\Temp\ipykernel_2604\2006770839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app["description"] = app["description"].apply(lambda x : re.sub(r'(【)(.*?)(】)','',x))#去除<.*>标签 ，应该还可以加一些括号类！！《
C:\Users\Justin\AppData\Local\Temp\ipykernel_2604\2006770839.py:4: SettingWithCo

In [25]:
app2.shape #某两百行变成了1800行

(3126, 4)